<a href="https://colab.research.google.com/github/ujjawal40/SEC-FinGraph-/blob/main/SEC_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
import os
import pandas as pd
import numpy as np
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase
import matplotlib.pyplot as plt
%matplotlib inline
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from sklearn.cluster import KMeans as SKMeans
from scipy import stats
from langchain.chains import GraphQAChain
from langchain.llms import OpenAI
import networkx as nx
from pyvis.network import Network


In [5]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
url = "neo4j+s://1c01afdf.databases.neo4j.io"
user = 'neo4j'
password = "Nr3rnqS47qcl0A_ls4ZZsMsZi2AGLTTMRZq6FThvVaw"
driver = GraphDatabase.driver(url, auth=(user,password))

In [ ]:
# COLUMN_MAP = {
#     "sub": ["adsh", "cik", "name", "filed", "period", "form"],
#     "num": ["adsh", "tag", "version", "ddate", "value",

#            ],
#     "tag": ["tag", "version", "tlabel",
#             # If your CSV has these, uncomment:
#             # "datatype", "iord", "crdr"
#            ],
#     "pre": ["adsh", "tag", "version", "plabel",
#             # If your CSV has these, uncomment:
#             # "stmt", "report", "line", "inpth", "negating"
#            ]
# }

# # Set batch size for processing rows in Neo4j
# BATCH_SIZE = 1000

# # Set maximum limits for nodes and edges to be ingested (each dataset is limited to 50,000 rows)
# LIMIT = 50000

# # Define a list of tags of interest (update these based on your analysis)
# tags_of_interest = [
#     "us-gaap:Revenues",
#     "us-gaap:NetIncomeLoss",
#     "us-gaap:Assets",
#     "us-gaap:Liabilities",
#     "us-gaap:LongTermDebt"  # Example for debt; adjust if needed
# ]

# def run_batches(session, query, df):
#     """
#     Splits the DataFrame `df` into batches of size BATCH_SIZE and
#     executes the given Cypher `query` for each batch.
#     """
#     records = df.to_dict("records")
#     for i in range(0, len(records), BATCH_SIZE):
#         batch = records[i:i+BATCH_SIZE]
#         session.run(query, {"batch": batch})

# def load_quarter(quarter):
#     """
#     Loads the data for a given quarter (q1, q2, q3, q4),
#     cleans it, filters it, and ingests it into Neo4j.
#     """
#     print(f"\n{'='*50}")
#     print(f"Processing 2018 {quarter.upper()}")
#     print(f"{'='*50}")


#     base_path = f"/content/drive/MyDrive/2018{quarter}/"

#     try:
#         # Read the datasets with the specified columns
#         sub = pd.read_csv(f"{base_path}sub.txt", delimiter='\t', usecols=COLUMN_MAP["sub"])
#         num = pd.read_csv(f"{base_path}num.txt", delimiter='\t', usecols=COLUMN_MAP["num"])
#         tag = pd.read_csv(f"{base_path}tag.txt", delimiter='\t', usecols=COLUMN_MAP["tag"])
#         pre = pd.read_csv(f"{base_path}pre.txt", delimiter='\t', usecols=COLUMN_MAP["pre"])

#         sub = sub.dropna(subset=["adsh", "cik"]).fillna({
#             'fp': 'N/A',
#             'afs': 'N/A',
#             'wksi': 0,
#             'prevrpt': 0,
#             'detail': 0,
#             'sic': 0
#         })
#         num = num.dropna(subset=["adsh", "tag", "version", "value"])
#         tag = tag.dropna(subset=["tag", "version"]).fillna({'crdr': 'U'})
#         pre = pre.dropna(subset=["adsh", "tag", "version"])

#         # Replace any remaining NaNs with None
#         sub = sub.replace({np.nan: None})
#         num = num.replace({np.nan: None})
#         tag = tag.replace({np.nan: None})
#         pre = pre.replace({np.nan: None})



#         # Limit the number of rows ingested from each dataset
#         sub = sub.head(LIMIT)
#         num = num.head(LIMIT)
#         tag = tag.head(LIMIT)
#         pre = pre.head(LIMIT)

#         # Open a session to Neo4j and create constraints
#         with driver.session() as session:
#             session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (s:Submission) REQUIRE s.adsh IS UNIQUE")
#             session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (t:Tag) REQUIRE (t.tag, t.version) IS UNIQUE")

#             # Insert Submission nodes
#             submission_query = """
#                 UNWIND $batch AS row
#                 MERGE (s:Submission {adsh: row.adsh})
#                 ON CREATE SET
#                     s.cik = row.cik,
#                     s.name = row.name,
#                     s.filed = row.filed,
#                     s.period = row.period,
#                     s.fp = coalesce(row.fp, 'N/A'),
#                     s.form = row.form,
#                     s.sic = coalesce(row.sic, 0),
#                     s.afs = coalesce(row.afs, 'N/A'),
#                     s.wksi = coalesce(row.wksi, 0),
#                     s.prevrpt = coalesce(row.prevrpt, 0),
#                     s.detail = coalesce(row.detail, 0)
#             """
#             run_batches(session, submission_query, sub)

#             # Insert Tag nodes
#             tag_query = """
#                 UNWIND $batch AS row
#                 MERGE (t:Tag {tag: row.tag, version: row.version})
#                 ON CREATE SET
#                     t.tlabel = row.tlabel,
#                     t.datatype = coalesce(row.datatype, 'N/A'),
#                     t.iord = coalesce(row.iord, 'N/A'),
#                     t.crdr = coalesce(row.crdr, 'U')
#             """
#             run_batches(session, tag_query, tag)


#             num_query = """
#                 UNWIND $batch AS row
#                 MATCH (s:Submission {adsh: row.adsh})
#                 MATCH (t:Tag {tag: row.tag, version: row.version})
#                 MERGE (f:NumericFact {
#                     adsh: row.adsh,
#                     tag: row.tag,
#                     version: row.version,
#                     ddate: row.ddate
#                 })
#                 ON CREATE SET
#                     f.value = toFloat(row.value),
#                     f.uom = coalesce(row.uom, 'N/A'),
#                     f.qtrs = coalesce(row.qtrs, 0),
#                     f.segments = coalesce(row.segments, 'N/A'),
#                     f.coreg = coalesce(row.coreg, 'N/A')
#                 MERGE (s)-[:HAS_FACT]->(f)
#                 MERGE (f)-[:USES_TAG]->(t)
#             """
#             run_batches(session, num_query, num)


#             pre_query = """
#                 UNWIND $batch AS row
#                 MATCH (s:Submission {adsh: row.adsh})
#                 MATCH (t:Tag {tag: row.tag, version: row.version})
#                 MERGE (p:Presentation {
#                     adsh: row.adsh,
#                     report: coalesce(row.report, 0),
#                     line: coalesce(row.line, 0)
#                 })
#                 ON CREATE SET
#                     p.stmt = coalesce(row.stmt, 'N/A'),
#                     p.plabel = row.plabel,
#                     p.inpth = coalesce(row.inpth, 'N/A'),
#                     p.negating = coalesce(row.negating, 'N/A')
#                 MERGE (s)-[:HAS_PRESENTATION]->(p)
#                 MERGE (p)-[:LINKS_TO_TAG]->(t)
#                 WITH p, row
#                 MATCH (f:NumericFact {adsh: row.adsh, tag: row.tag, version: row.version})
#                 MERGE (p)-[:REFERENCES_FACT]->(f)
#             """
#             run_batches(session, pre_query, pre)

#         print(f" 2018 {quarter.upper()} loaded successfully!")

#     except Exception as e:
#         print(f"  Error processing {quarter.upper()}: {str(e)}")

# for quarter in ["q1", "q2", "q3", "q4"]:
#     load_quarter(quarter)

# print("\n All data loaded! Validate with a Cypher query, e.g.:")
# print("MATCH (s:Submission) RETURN s.adsh, s.form, s.name LIMIT 10")


In [9]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [12]:
query = "MATCH (s:Submission)-[r1:HAS_FACT]->(f:NumericFact)-[r2:USES_TAG]->(t:Tag) RETURN s, r1, f, r2, t LIMIT 50"

g = Neo4jGraphWidget(driver)

g.show_cypher(query)

GraphWidget(layout=Layout(height='800px', width='100%'))

In [15]:
query4 = """
MATCH p = (s:Submission)-[:HAS_PRESENTATION]->(pr:Presentation)-[:LINKS_TO_TAG]->(t:Tag)
RETURN p
LIMIT 50
"""
g.show_cypher(query4)

GraphWidget(layout=Layout(height='800px', width='100%'))

In [16]:
visual_query = """
MATCH p = (s:Submission {adsh: "0001193125-18-020970"})-[*..2]-(x)
RETURN p
LIMIT 50
"""

g.show_cypher(visual_query)

GraphWidget(layout=Layout(height='500px', width='100%'))